<a href="https://colab.research.google.com/github/JorgeLGomes/PROJETA/blob/main/Projeta_API_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=darkblue>PROJEÇÕES DE MUDANÇA DO CLIMA PARA A AMÉRICA DO SUL REGIONALIZADAS PELO MODELO ETA

# <font color=red>**INFORMAÇÕES**


*   **Como Rodar**
       O simbolo de play (▶), no canto superior esquerdo de cada célula, executa o cógigo contido na mesma.
       Sempre que carregar o código no google colab, deve ser executado cada célula na sequência (**<font color=green>Carregamento das Bibliotecas, Definição das Funções e Seleção dos Dados** <font color=black>)
Ao finalizar a execução de todas as células, uma consulta terá sido feita no banco de dados e um gráfico, do dado adquirido, será plotado no final da página. Essa ação somente irá gerar o gráfico da informação, nenhum dado será salvo. Para que as informações sejam salvas no disco, devem ser habilitadas as opções: 

    Salva_CSV, para salvar os dados no formato csv 

    Salva_Grafico, para salvar o gráfico no formato png 
    
    Após a primeira execução, enquanto o google colab não for desconectado, não será preciso executar as duas primeiras células (**<font color=green>Carregamento das Bibliotecas, Definição das Funções** <font color=black>). Ao fazer Uma nova seleção, basta executar o  play (▶) para uma nova consulta ao banco. (OBS: foi definido para que a célula (**<font color=green>Seleção dos Dados** <font color=black>) seja executada automáticamente após uma seleção, caso não ocorra, basta clicar no play)
*   **Opções**
    *   Rodada_selecionada
        
        Seleção da rodada/cenário
    *   Frequencia_temporal
        Frequencia temporal de obtenção dos dados. As requisições ficam mais demoradas com o aumento da frequência 
    *   model_var
        
        Variáveis disponibilizadas
    *   Latitude
        
        Campo para a seleção da latitude do ponto desejado. 
        
        A disponibilização dos dados é dependente da rodada, segue abaixo o range onde os dados estão disponíveis
        *   continental: range disponível entre 50S a 27N
        *   sudeste-BR:  range disponível entre 26S a 18S
    *   Longitude
        
        Campo para a seleção da longitude do ponto desejado. 
        
        A disponibilização dos dados é dependente da rodada, segue abaixo o range onde os dados estão disponíveis
        *   continental: range disponível entre 99W a 32W
        *   sudeste-BR:  range disponível entre 50W a 41W
    *   Salva_CSV: 
    
        habilita o código a salvar a informação adquirida no banco no formato csv
    *   Plota_Grafico: 
    
        habilita o código a plotar a informação adquirida em um gráfico de linhas
    *   Salva_Grafico: 
    
        habilita o código a salvar a o gráfico em formato png. Essa opção só será efetiva se a opção Plota_Gráfico estiver selecionada

**Dica**

Caso as seleções Salva_CSV e Salva_Grafico estiverm selecionadas, uma dica é desabilitar, nas configurações do navegador, a opção de "Perguntar onde salvar cada arquivo antes de fazer download", com isso, a cada seleção, os arquivos serão salvos automaticamente.  

Para informações sobre o google colab: 
https://research.google.com/colaboratory/intl/pt-BR/faq.html

In [ ]:
#@title Carregamento das Bibliotecas
from ipywidgets import interact, Layout
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import clear_output
import scipy as sci
import datetime
import json
import requests
import csv
from google.colab import files
import altair as alt

In [ ]:
#@title Definição das Funções { run: "auto" }
def set_param_cenario(Rodada_selecionada):
  global model, resol, Cenario, Dominio, ModDrive,yyyymmdd_i,yyyymmdd_f,latlon_extr
  if Rodada_selecionada == '20km, RCP4.5, continental, MIROC5': 
    model="1"
    yyyymmdd_i="2006-01-01"
    yyyymmdd_f="2099-12-31"
    resol="20km"
    Cenario="RCP4.5"
    Dominio="continental"
    ModDrive="MIROC5"
    latlon_extr=1
  elif Rodada_selecionada == '20km, RCP8.5, continental, MIROC5':
    model="2" 
    yyyymmdd_i="2006-01-01"
    yyyymmdd_f="2099-12-31"
    resol="20km"
    Cenario="RCP8.5"
    Dominio="continental"
    ModDrive="MIROC5"
    latlon_extr=1
  elif Rodada_selecionada == '20km, Historico, continental, MIROC5':
    model="15" 
    yyyymmdd_i="1961-01-01"
    yyyymmdd_f="2005-12-31"
    resol="20km"
    Cenario="Historico"
    Dominio="continental"
    ModDrive="MIROC5"
    latlon_extr=1
  elif Rodada_selecionada == '05 km, RCP4.5, sudeste-BR, HADGEM2-ES':
    model="4"
    yyyymmdd_i="2006-01-01"
    yyyymmdd_f="2099-12-31"
    resol="05km"
    Cenario="RCP4.5"
    Dominio="sudeste-BR"
    ModDrive="HADGEM2-ES"
    latlon_extr=1
  elif Rodada_selecionada == '05 km, RCP8.5, sudeste-BR, HADGEM2-ES':
    model="6"
    yyyymmdd_i="2006-01-01"
    yyyymmdd_f="2099-12-31"
    resol="05km"
    Cenario="RCP8.5"
    Dominio="sudeste-BR"
    ModDrive="HADGEM2-ES"
    latlon_extr=1
  elif Rodada_selecionada == '05 km, Historico, sudeste-BR, HADGEM2-ES':
    model="16"
    yyyymmdd_i="1961-01-01"
    yyyymmdd_f="2005-12-31"
    resol="05km"
    Cenario="Historico"
    Dominio="sudeste-BR"
    ModDrive="HADGEM2-ES"
    latlon_extr=1
  return 

def set_param_freq(Frequencia_temporal):
  global model_freq, freq_idx
  if Frequencia_temporal == 'ANUAL': 
    model_freq="YEARLY"
    freq_idx="4"
  elif Frequencia_temporal == 'MENSAL':
    model_freq="MONTHLY" 
    freq_idx="3"
  elif Frequencia_temporal == 'DIARIA':
    model_freq="DAILY" 
    freq_idx="2"
  elif Frequencia_temporal == '3 HORAS':
    model_freq="HOURLY"
    freq_idx="1"
  return
def set_variavel(model_var):
  global yfig_title
  if model_var == 'PREC':
    yfig_title="Precipitação (mm)"
  elif model_var == 'TP2M':
    yfig_title="Temperatura 2m (C)"
  elif model_var == 'PRCPTOT':
    yfig_title="Precipitação (C)"
  elif model_var == 'TSFC':
    yfig_title="Temperatura superficie (C)"

def Write_CSV():
    df.to_csv(file_out+'.csv',sep=';',index=False)
    files.download(file_out+'.csv')
    return

def Gera_URL():
    global url,file_out
    url="https://projeta.cptec.inpe.br/api/v1/public/ETA/"+model+"/"
    url=url+model_freq+"/"+freq_idx+"/"+datei[1]+"/"+datei[0]
    url=url+"/"+datef[1]+"/"+datef[0]+"/"+model_var+"/"+str(Latitude)+"/"+str(Longitude)+"/"
    file_out="Eta"+resol+"_"+Cenario+"_"+Dominio+"_"+ModDrive+"_"+model_freq+"_"+datei[0]+"-"+datei[1]+"_"+datef[0]+"-"+datef[1]+"_"+model_var+"_"+str(Latitude)+"_"+str(Longitude)
    return
def Get_Data():
    global df
    request = requests.get(url)
    request_json = request.json()
    df = pd.json_normalize(request_json)
    return 
def on_button_clicked_save_fig(b):
    plt.savefig(file_out+'.png')
    files.download(file_out+'.png')
    return
def Save_Fig():
    button = widgets.Button(description="Save Fig")
    display(button)
    button.on_click(on_button_clicked_save_fig)
    return

def Plota(Salva_Grafico):
#    %matplotlib notebook
    %matplotlib inline
    Gera_URL()
    #%config InlineBackend.figure_format ='retina'
    df.plot(x ='date', y='value', kind='line')
    plt.title(file_out, size = 10)
    plt.xlabel("Data")
    plt.ylabel(yfig_title)
    if Salva_Grafico:
      plt.savefig(file_out+'.png')
      files.download(file_out+'.png')
    plt.show()
    return

In [ ]:
#@title Seleção dos Dados
#@title  { run: "auto" }
#@title  { run: "auto" }
#@title  { run: "auto" }
#@title 
#@title  { run: "auto", display-mode: "form" }
#@title  { run: "auto" }
#@title  { form-width: "50%" }
#@title  { form-width: "50%", display-mode: "both" }

Rodada_selecionada = "20km, Historico, continental, MIROC5" #@param ["20km, RCP4.5, continental, MIROC5", "20km, RCP8.5, continental, MIROC5", "20km, Historico, continental, MIROC5", "05 km, RCP8.5, sudeste-BR, HADGEM2-ES", "05 km, Historico, sudeste-BR, HADGEM2-ES"] {allow-input: true}
set_param_cenario(Rodada_selecionada)

datei=yyyymmdd_i.split("-")
datef=yyyymmdd_f.split("-")
Frequencia_temporal = "MENSAL" #@param ["ANUAL", "MENSAL", "DIARIA", "3 HORAS"] {allow-input: true}
set_param_freq(Frequencia_temporal)

model_var = "TP2M"  #@param ['PREC', 'TP2M', 'PRCPTOT', 'TSFC']{allow-input: true}
set_variavel(model_var)
Latitude = 13.15 #@param {type:"slider", min: -49, max:26, step:0.01}
Longitude = -50.15 #@param {type:"slider", min:-99, max:-32, step:0.01}
Salva_CSV = False #@param {type:"boolean"}
Plota_Grafico = True #@param {type:"boolean"}
Salva_Grafico = False #@param {type:"boolean"}

#Latitude = -25 #@param {type:"number"}
#Longitude = -50 #@param {type:"number"}


Gera_URL()
print(url)
Get_Data()
if Plota_Grafico:
  Plota(Salva_Grafico)
if Salva_CSV:
  Write_CSV()

# **URL e Gráfico do Dado Selecionado**

#<font color=red><p align="justify">**Quando os dados forem utilizados para trabalhos acadêmicos e publicações, utilizar nas referências bibliográficas a fonte dos dados CPTEC/INPE da seguinte forma "Dados gerados pelo CPTEC/INPE e disponibilizados na Plataforma PROJETA" e as referências bibliográficas:**</p>

<p align="justify">Chou, S.C, Lyra, A. , Mourão, C. , Dereczynski, C. , Pilotto, I. , Gomes, J. , Bustamante, J. , Tavares, P. , Silva, A. , Rodrigues, D. , Campos, D. , Chagas, D. , Sueiro, G. , Siqueira, G. , Nobre, P. and Marengo, J. (2014) Evaluation of the Eta Simulations Nested in Three Global Climate Models. American Journal of Climate Change, 3, 438-454. doi:10.4236/ajcc.2014.35039. http://www.scirp.org/journal/PaperInformation.aspx?PaperID=52887#.VakHg_lViko

Chou, S.C, Lyra, A. , Mourão, C. , Dereczynski, C. , Pilotto, I. , Gomes, J. , Bustamante, J. , Tavares, P. , Silva, A. , Rodrigues, D. , Campos, D. , Chagas, D. , Sueiro, G. , Siqueira, G. and Marengo, J. (2014) Assessment of Climate Change over South America under RCP 4.5 and 8.5 Downscaling Scenarios. American Journal of Climate Change,3, 512-527. doi: 10.4236/ajcc.2014.35043. http://www.scirp.org/journal/PaperInformation.aspx?PaperID=52877#.VakIh_lVikp

Lyra, A., Tavares, P., Chou, S.C., Sueiro, G., Dereczynski, C.P., Sondermann, M., Silva, A., Marengo, J., Giarolla, A. 2017. Climate change projections over three metropolitan regions in Southeast Brazil using the non-hydrostatic Eta regional climate model at 5-km resolution Theor Appl Climatol. doi:10.1007/s00704-017-2067-z. https://link.springer.com/article/10.1007/s00704-017-2067-z

Terceira Comunicação Nacional do Brasil à Convenção-Quadro das Nações Unidas sobre Mudança do Clima</p>

Executive Summary: http://unfccc.int/resource/docs/natc/branc3es.pdf

Volume 1: http://unfccc.int/resource/docs/natc/branc3v1.pdf

Volume 2: http://unfccc.int/resource/docs/natc/branc3v2.pdf 

Volume 3: http://unfccc.int/resource/docs/natc/branc3v3.pdf